In [1]:
# playing around with the data sliceing it to "train" and "test"

In [2]:
import pandas as pd
train_df = pd.read_excel('appDescriptions.xlsx', sheetname="Examples")

#print(train_df.description)
train_df.description.replace({r'\W+':' '}, regex=True, inplace=True)
#print(train_df.description)
train_df.description.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
#print(train_df.description)

from sklearn.model_selection import train_test_split
train, test = train_test_split(train_df, test_size=0.2)

In [4]:
# Extracting features from text files
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train.description.values)
X_train_counts.shape

(1195, 35616)

In [5]:
# TF-IDF helps to give more score to "meaningful words" 
# and less scores to words that are getting repeated (in our case: app, android, ios, etc..)

In [6]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(1195, 35616)

In [7]:
# Trying NB clssifier

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),])
text_clf = text_clf.fit(train.description.values, train.segment.values)

In [10]:
# not so well :-(

In [20]:
import numpy as np
predicted = text_clf.predict(test.description.values)
#print(predicted)
np.mean(predicted == test.segment.values)

0.3612040133779264

In [22]:
from sklearn import metrics
print(metrics.classification_report(test.segment.values, predicted))

                                precision    recall  f1-score   support

                 Adult Content       0.00      0.00      0.00         1
          Arts & Entertainment       0.71      0.17      0.28        29
           Career & Employment       0.00      0.00      0.00         1
                     Education       0.00      0.00      0.00         5
              Fashion & Beauty       0.00      0.00      0.00         2
                       Finance       0.67      0.64      0.65        28
                  Food & Drink       0.00      0.00      0.00         9
                         Games       1.00      0.21      0.35        19
        Health & Personal care       0.00      0.00      0.00        18
                       Hobbies       0.67      0.24      0.35        25
                 Home & Garden       0.00      0.00      0.00         1
                    Lifestyles       0.00      0.00      0.00         7
         News & Current Events       0.00      0.00      0.00  

In [12]:
# Trying SVM classifier (with stop words)

In [23]:
text_clf_svm = Pipeline([('vect', CountVectorizer(stop_words='english')),
                      ('tfidf', TfidfTransformer()),
                    ('clf-svm', SGDClassifier(loss='modified_huber', penalty='l2',alpha=0.003, n_iter=5, random_state=30))])
text_clf_svm = text_clf_svm.fit(train.description.values, train.segment.values)

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [ ]:
# This time it's better

In [24]:
import numpy as np
predicted = text_clf_svm.predict(test.description.values)
#print(predicted)
np.mean(predicted == test.segment.values)

0.65217391304347827

In [25]:
from sklearn import metrics
print(metrics.classification_report(test.segment.values, predicted))

                                precision    recall  f1-score   support

                 Adult Content       0.00      0.00      0.00         1
          Arts & Entertainment       0.60      0.41      0.49        29
           Career & Employment       0.00      0.00      0.00         1
                     Education       0.00      0.00      0.00         5
              Fashion & Beauty       0.00      0.00      0.00         2
                       Finance       0.72      0.75      0.74        28
                  Food & Drink       0.83      0.56      0.67         9
                         Games       0.65      0.79      0.71        19
        Health & Personal care       1.00      0.83      0.91        18
                       Hobbies       0.60      0.60      0.60        25
                 Home & Garden       0.00      0.00      0.00         1
                    Lifestyles       0.75      0.86      0.80         7
         News & Current Events       0.67      0.60      0.63  